In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [5]:
ckd_data = pd.read_csv("ckd_data.csv", error_bad_lines=False, sep=r'\s*,\s*')
ckd_data.columns = [col[1:-1] for col in ckd_data.columns]
ckd_data = ckd_data.iloc[:, :-1]

<ipython-input-5-7118ab3df0c9>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ckd_data = pd.read_csv("ckd_data.csv", error_bad_lines=False, sep=r'\s*,\s*')


In [6]:
ckd_data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48,80,1.02,1,0,?,normal,notpresent,notpresent,121,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,?,normal,notpresent,notpresent,?,...,38,6000,?,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [7]:
print(ckd_data.loc[[66]])

   age  bp    sg al su       rbc      pc         pcc          ba  bgr  ...  \
66  67  70  1.02  2  0  abnormal  normal  notpresent  notpresent  150  ...   

   pcv wbcc rbcc  htn   dm cad appet   pe ane class  
66   ?    ?    ?  yes  yes  no  good  yes  no   ckd  

[1 rows x 25 columns]


In [37]:
ckd_data = ckd_data.replace(to_replace='?', value='Nan')
print(ckd_data.loc[66])

age              67
bp               70
sg             1.02
al                2
su                0
rbc        abnormal
pc           normal
pcc      notpresent
ba       notpresent
bgr             150
bu               55
sc              1.6
sod             131
pot             4.8
hemo            Nan
pcv             Nan
wbcc            Nan
rbcc            Nan
htn             yes
dm              yes
cad              no
appet          good
pe              yes
ane              no
class           ckd
Name: 66, dtype: object


In [9]:
ckd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   age     400 non-null    object
 1   bp      400 non-null    object
 2   sg      400 non-null    object
 3   al      400 non-null    object
 4   su      400 non-null    object
 5   rbc     400 non-null    object
 6   pc      400 non-null    object
 7   pcc     400 non-null    object
 8   ba      400 non-null    object
 9   bgr     400 non-null    object
 10  bu      400 non-null    object
 11  sc      400 non-null    object
 12  sod     400 non-null    object
 13  pot     400 non-null    object
 14  hemo    400 non-null    object
 15  pcv     400 non-null    object
 16  wbcc    400 non-null    object
 17  rbcc    400 non-null    object
 18  htn     400 non-null    object
 19  dm      399 non-null    object
 20  cad     400 non-null    object
 21  appet   400 non-null    object
 22  pe      400 non-null    ob

In [10]:
from sklearn.model_selection import train_test_split
ckd_train_data, ckd_test_data = train_test_split(ckd_data, test_size=0.2, random_state=42)

In [11]:
# Separate out the labels
ckd_train_data_label = ckd_train_data['class'].copy()
ckd_test_data_label = ckd_test_data['class'].copy()
ckd_train_data_label.head()

3         ckd
18        ckd
202       ckd
250    notckd
274    notckd
Name: class, dtype: object

In [12]:
# List of Features only dataset. Drop the labels
ckd_train_data = ckd_train_data.drop("class", axis=1)
ckd_test_data = ckd_test_data.drop("class", axis=1)
ckd_train_data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,hemo,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,11.2,32,6700,3.9,yes,no,no,poor,yes,yes
18,60,100,1.025,0,3,Nan,normal,notpresent,notpresent,263,...,12.7,37,11400,4.3,yes,yes,yes,good,no,no
202,78,60,Nan,Nan,Nan,Nan,Nan,notpresent,notpresent,114,...,8,24,Nan,Nan,no,yes,no,good,no,yes
250,40,80,1.025,0,0,normal,normal,notpresent,notpresent,140,...,15,48,10400,4.5,no,no,no,good,no,no
274,19,80,1.02,0,0,normal,normal,notpresent,notpresent,107,...,14.4,44,Nan,Nan,no,no,no,good,no,no


In [13]:
def impute_nan_add_variable(DataFrame,ColName):
    #1. add new column and replace if category is null then 1 else 0
    DataFrame[ColName+"_Imputed"] = np.where(DataFrame[ColName].isnull(),1,0)
    
    # 2. Take most occured category in that vairable (.mode())
    Mode_Category = DataFrame[ColName].mode()[0]
    
    ## 2.1 Replace NAN values with most occured category in actual vairable
    DataFrame[ColName].fillna(Mode_Category,inplace=True)

In [14]:
# Call function to impute NAN values for categorical variables and add new importance feature
for Columns in ['rbc','pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']:
    impute_nan_add_variable(ckd_train_data,Columns)

In [15]:
ckd_train_data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,rbc_Imputed,pc_Imputed,pcc_Imputed,ba_Imputed,htn_Imputed,dm_Imputed,cad_Imputed,appet_Imputed,pe_Imputed,ane_Imputed
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,0,0,0,0,0,0,0,0,0,0
18,60,100,1.025,0,3,Nan,normal,notpresent,notpresent,263,...,0,0,0,0,0,0,0,0,0,0
202,78,60,Nan,Nan,Nan,Nan,Nan,notpresent,notpresent,114,...,0,0,0,0,0,0,0,0,0,0
250,40,80,1.025,0,0,normal,normal,notpresent,notpresent,140,...,0,0,0,0,0,0,0,0,0,0
274,19,80,1.02,0,0,normal,normal,notpresent,notpresent,107,...,0,0,0,0,0,0,0,0,0,0


In [16]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scalar', StandardScaler()),
])

In [17]:
ckd_train_data_numeric = ckd_train_data.drop(['rbc','pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane'], axis=1)

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

num_attribs = list(ckd_train_data_numeric)
cat_attribs = ['rbc','pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OrdinalEncoder(), cat_attribs),
])

In [21]:
ckd_train_data_prepared = full_pipeline.fit_transform(ckd_train_data)
print(ckd_train_data_prepared[:5])

[[-0.1699142  -0.41503806 -2.32419215  2.41090936 -0.3775227  -0.40955203
   0.02883762  0.18473616 -2.7211447  -0.68100449 -0.53127344 -0.95717839
  -0.63259935 -1.04249276  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   2.          1.          2.          1.          2.          1.
   1.          1.          1.          1.        ]
 [ 0.52126221  1.75037792  1.32159946 -0.68123892  2.56899591  1.56244333
  -0.56414514 -0.27602434 -0.26352239 -0.10466216  0.03291074 -0.32407156
   1.19953896 -0.55172751  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          2.          1.          1.          2.          2.
   2.          0.          0.          0.        ]
 [ 1.55802683 -1.13684339  0.41015156 -0.68123892 -0.3775227  -0.45007249
   0.39689588  0.01886238 -0.26352239  0.40764212 -1.73486637 -1.97014931
  -0.10634686  0.06172907  0.          0.          0.     

In [23]:
ckd_train_data_binary_labels = (ckd_train_data_label == 'ckd')
print(ckd_train_data_binary_labels[:5])

3       True
18      True
202     True
250    False
274    False
Name: class, dtype: bool


In [24]:
from sklearn.svm import LinearSVC
svm_clf1 = LinearSVC(C=1, loss="hinge", random_state=42)
svm_clf1.fit(ckd_train_data_prepared, ckd_train_data_binary_labels)

LinearSVC(C=1, loss='hinge', random_state=42)

In [26]:
from sklearn.svm import SVC
rbf_kernel_svm_clf = SVC(kernel="rbf", gamma=5, C=0.001)
rbf_kernel_svm_clf.fit(ckd_train_data_prepared, ckd_train_data_binary_labels)

svm_clf1 = SVC(kernel="linear", C=1)
svm_clf1.fit(ckd_train_data_prepared, ckd_train_data_binary_labels)

SVC(C=1, kernel='linear')

In [28]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(ckd_train_data_prepared, ckd_train_data_binary_labels)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1)

In [29]:
# Lets evaluate the cross validation to compare the models performance on this
# binary classification problem. 
from sklearn.model_selection import cross_val_predict
ckd_train_data_LSVC_predictions = cross_val_predict(svm_clf1, ckd_train_data_prepared, ckd_train_data_binary_labels, cv=3)
ckd_train_data_rbf_kernel_SVM_predictions = cross_val_predict(rbf_kernel_svm_clf, ckd_train_data_prepared, ckd_train_data_binary_labels, cv=3)
ckd_train_data_RF_predictions = cross_val_predict(rnd_clf, ckd_train_data_prepared, ckd_train_data_binary_labels, cv=3)

In [30]:
# Comparing the confusion matrix of the 3 algorithm options
from sklearn.metrics import confusion_matrix
confusion_matrix(ckd_train_data_binary_labels, ckd_train_data_LSVC_predictions)

array([[121,   1],
       [  3, 195]])

In [31]:
confusion_matrix(ckd_train_data_binary_labels, ckd_train_data_rbf_kernel_SVM_predictions)

array([[  0, 122],
       [  0, 198]])

In [38]:
confusion_matrix(ckd_train_data_binary_labels, ckd_train_data_RF_predictions)

array([[120,   2],
       [  0, 198]])

In [54]:
from sklearn.metrics import f1_score
f1_score(ckd_train_data_binary_labels, ckd_train_data_LSVC_predictions)

0.9898477157360407

In [55]:
f1_score(ckd_train_data_binary_labels, ckd_train_data_rbf_kernel_SVM_predictions)

0.7644787644787645

In [56]:
f1_score(ckd_train_data_binary_labels, ckd_train_data_RF_predictions)

0.9949748743718593

In [39]:
#Prepare the test data
for Columns in ['rbc','pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']:
    impute_nan_add_variable(ckd_test_data,Columns)

In [73]:
# Execute the pipeline on the test data
ckd_test_data_prepared = full_pipeline.fit_transform(ckd_test_data)

In [74]:
# Prediction using the Linear SVC
ckd_test_data_Linear_SVC_predictions = svm_clf1.predict(ckd_test_data_prepared)

In [84]:
# Prediction using the RBF Kernel SVM
ckd_test_data_rbf_kernel_SVM_predictions = rbf_kernel_svm_clf.predict(ckd_test_data_prepared)

In [85]:
# Predictions using Random Forest Classifier
ckd_test_data_RF_predictions = rnd_clf.predict(ckd_test_data_prepared)

In [86]:
ckd_test_data_RF_predictions

array([ True, False,  True,  True,  True,  True, False,  True, False,
        True,  True, False,  True,  True,  True,  True, False, False,
        True, False,  True,  True, False, False,  True, False,  True,
        True, False,  True, False,  True, False,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False, False,  True,
       False,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True, False, False, False,  True, False,  True,
        True,  True, False,  True,  True, False,  True, False])

In [87]:
# Test predictions metrics
ckd_test_data_binary_labels = (ckd_test_data_label == 'ckd')

array([[28,  0],
       [ 0, 52]])

In [88]:
# Confusion matrix of Linear SVC predictions
confusion_matrix(ckd_test_data_binary_labels, ckd_test_data_Linear_SVC_predictions)

array([[27,  1],
       [ 0, 52]])

In [89]:
f1_score(ckd_test_data_binary_labels, ckd_test_data_Linear_SVC_predictions)

0.9904761904761905

In [90]:
# Confusion matrix of RBF Kernel predictions
confusion_matrix(ckd_test_data_binary_labels, ckd_test_data_rbf_kernel_SVM_predictions)

array([[ 0, 28],
       [ 0, 52]])

In [91]:
f1_score(ckd_test_data_binary_labels, ckd_test_data_rbf_kernel_SVM_predictions)

0.787878787878788

In [92]:
# Confusion matrix of Random Forest predictions
confusion_matrix(ckd_test_data_binary_labels, ckd_test_data_RF_predictions)

array([[28,  0],
       [ 0, 52]])

In [82]:
f1_score(ckd_test_data_binary_labels, ckd_test_data_RF_predictions)

1.0